In [1]:
import pandas as pd
import numpy as np
import re
from shapely.geometry import Point, Polygon
import geopandas as gpd



In [2]:
def to_float(i):
    """
    el parámetro i es un del tipo str
    -----------------------------------
    La función to_float convierte a 'i' a tipo flotante
    -----------------------------------
    devuelve un valor del tipo float
    
    """
    try:
        i = float(i)
        return i
    except:
        return None

In [3]:
def transform(row):
    """
    el parámetro row es un valor del tipo str
    ----------------------------------------------
    separa los elementos de cada row de acuerdo a ciertos caracteres
    depura aquellos que no cumplen ciertas condiciones asignandoles el valor de lista vacia
    transforma cada cadena de texto a valores del tipo float
    ------------------------------------------------
    Retorna coor la cual puede ser una lista con valores flotantes 
    o una lista vacía.

    """
    input = row.split(',')

    size_list = len(input)


    if size_list ==1:
        coor = None
    elif size_list ==2:
        try:
            pre_punto = map(to_float,input)
            coor = list(pre_punto)
            if None in coor:
                coor = None
                return coor
            else:
                return coor
        except ValueError as e:
            print(e)
            return None
    elif size_list >2:
        not_point = re.split(r",0\\n\\n|,0|0\s-|,",row)
        if ("" in not_point):
            not_point.remove("")
            list_with_float = map(to_float,not_point)
            coor = list(list_with_float)
            return coor
        else:
            list_with_float = map(to_float,not_point)
            coor = list(list_with_float)
            return coor



In [4]:
def to_matrix(listado):
    """
    el parámetro listado es un objeto del tipo lista
    -----------------------------------
    transforma una lista a un array de numpy
    asigna la forma del array
    -----------------------------------
    devuelve una matriz
    """
    n = len(listado)
    c = 2
    f =  n/c
    f = int(f)
    vector = np.array(listado)
    matriz = vector.reshape(f,c)
    return matriz

In [5]:
def geometry_apply(x):
    """
    el parámetro x es una lista
    ------------------------------------------
    de acuerdo a la longitud de la lista aplica una o dos funciones
    -----------------------------------------
    devuelve geometry que puede ser un objeto Point, Polygon o un valor None
    """
    if len(x) == 2:
        try:
            orden = sorted(x)
            geometry = Point(orden)
            return geometry
        except TypeError as e:
            pass
    elif len(x) >5 and len(x) %2 == 0:
        return Polygon(to_matrix(x))
    else:
        geometry = None
        print(geometry)
        return geometry


In [6]:
def typeshape(x):
    """
    El parámetro x es un str
    ----------------------------------------
    De acuerdo al valor genera ciertas cadenas de texto
    ---------------------------------------
    retorna label que es un str
    """
    n=x
    if int(n) == 2:
        label = "Point"
    else:
        label = "Polygon"
    return label    

In [7]:
def main(file_input):
    """
    file_input recibe al archivo Sitios.txt
    -----------------------------------------------
    Lee el archivo
    genera columnas y aplica funciones
    crea 3 subsets distintos
    elimina columnas temporales y columna Coordenadas areas
    exportar los sets a un formato específico
    -----------------------------------------------
    genera los archivos de salida o un error
    """
    try:
        sitios= pd.read_csv(file_input)
        sitiosbd = sitios.copy()
        sitiosbd['info_geo'] = sitiosbd['Coordenadas.areas']
        sitiosbd['info_geo']=sitiosbd['info_geo'].astype(str)
        sitiosbd['info_geo']= sitiosbd['info_geo'].apply(transform)
        desechados=sitiosbd[sitiosbd['info_geo'].isnull()]
        sitiosbd_depurate=sitiosbd[sitiosbd['info_geo'].notna()]
        sitiosbd_depurate['n_vertices'] = sitiosbd_depurate['info_geo'].apply(lambda x:len(x))
        # sitiosbd_depurate.to_csv(f'{PATH}n_vertices.csv')
        # sitiosbd_depurate['type_geometry'] = sitiosbd_depurate['info_geo'].apply(arreglo)
        sitiosbd_depurate['type_geometry'] = sitiosbd_depurate['info_geo'].apply(geometry_apply)
        sitiosbd_depurate=sitiosbd_depurate[sitiosbd_depurate['type_geometry'].notna()]
        desechados_2=sitiosbd_depurate[sitiosbd_depurate['type_geometry'].isnull()]
        desechados = pd.concat([desechados_2,desechados],axis=0)
        desechados.to_csv('./errors.csv', index=False)
        sitiosbd_depurate['type_shape'] = sitiosbd_depurate['n_vertices'].apply(typeshape)
        sitiosbd_polygon = sitiosbd_depurate[sitiosbd_depurate['type_shape']=='Polygon']
        sitiosbd_point = sitiosbd_depurate[sitiosbd_depurate['type_shape']=='Point']
        gdf_points_sitios = gpd.GeoDataFrame(sitiosbd_point,geometry='type_geometry', crs="EPSG:4326")
        gdf_polygons_sitios = gpd.GeoDataFrame(sitiosbd_polygon,geometry='type_geometry', crs="EPSG:4326")
 #el formato GeoJSON no soporta el formato de listas(un arreglo)
        eliminar = ['Coordenadas.areas','info_geo','n_vertices','type_shape','Nombre.Productor','Bitacora.registrada.por']
        gdf_points_sitios=gdf_points_sitios.drop(columns=eliminar)
        gdf_polygons_sitios=gdf_polygons_sitios.drop(columns=eliminar)

        gdf_points_sitios.to_file('./puntos.geojson', driver='GeoJSON')
        gdf_polygons_sitios.to_file('./poligonos.geojson', driver='GeoJSON')
    except FileNotFoundError as e:
        print(f'Por favor ejecute el script en el mismo directorio donde se encuentre el archivo Sitio.txt {e}')

In [8]:
if __name__ == "__main__":
    main('../data/bitacoras_corte_julio.csv')

/tmp/ipykernel_22445/4100383097.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sitiosbd_depurate['n_vertices'] = sitiosbd_depurate['info_geo'].apply(lambda x:len(x))
/tmp/ipykernel_22445/4100383097.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sitiosbd_depurate['type_geometry'] = sitiosbd_depurate['info_geo'].apply(geometry_apply)
